# Import needed library

In [1]:
import pandas as pd 
import numpy as np 
import tmdbv3api as tmdb 

# Data Gathering
- Dữ liệu thu thập tới từ list film của Anh, Mỹ, Nhật, Hàn, Trung quốc, Việt Nam, Nga, Pháp, Đức
- Tên phim theo các năm được lấy từ wikipedia, dữ liệu từ APi tmdb
- Thời gian từ 2020s - 1990s 

In [2]:
from tmdbv3api import TMDb, Movie, Search
from os import getenv
tmdb = TMDb()

tmdb.api_key = getenv('TMDB_API_KEY')
search = Search()

## Korean 

- Dữ liệu dạng bảng từ 2007 - 2019 
- Dữ liệu từ 2007 - 2008: Cột en-title trùng với kor-title 

In [7]:
from typing import List
pre_link = 'https://en.wikipedia.org/wiki/List_of_Japanese_films_of_'



def title_handle(lst: List): 
    res = ''
    for i in lst[:-1]: 
        res += i  + ' '

    return res
"""
- Hàm lấy dữ liệu film từ wiki : 
- Hàm trả về en-title + năm của phim 
- Đầu vào là năm  
"""

def get_films_on_wiki(year: int): 
    link = pre_link + str(year)

    page_tables = pd.read_html(link, header= None)

    sub_df = page_tables[2]
    res = sub_df[['Opening', 'Title']].copy()
    res.rename(columns={'Opening': 'Released', 'Title': 'English title'}, inplace=True)
    # xử lý title của film
    # if year == 2007 or year == 2008: 
    #     sub_df['English titles'] = sub_df['English/Korean Title'].str.split(r'([a-zA-Z]+)')
    #     sub_df['English title'] = sub_df['English titles'].apply(lambda x: title_handle(x))

    
    res.dropna(inplace=True)
    return res


    

In [4]:
dct = {
    'january' : 1, 
    'february' : 2,
    'march' : 3, 
    'april' :  4,
    'may' : 5, 
    'june' : 6, 
    'july' : 7, 
    'august' : 8, 
    'september' : 9, 
    'october' : 10, 
    'november' : 11, 
    'december' : 12
}
# thay đổi year 
def change_year_type(x,  year: int): 
    num_month = dct[x[1]]
    return str(year)  + '-' + str(num_month) + '-' + x[0]

# thay đổi cột released trong bảng 
def change_release_year(df: pd.DataFrame, year: int): 
    df['Released'] = df['Released'].str.lower()

    df[['Day', 'Month', 'Ex']] = df['Released'].str.extract(r'(\d+)\s+(\w+)(\[.*\])*')
    

    df['Day'] = df['Day'].astype('str')
    df['Released'] = df[['Day', 'Month']].apply(lambda x: change_year_type(x, year), axis = 1)
    df['Released'] = pd.to_datetime(df['Released'])
    df.drop(['Day', 'Month', 'Ex'], axis = 1, inplace = True)
    
    return df

# thêm thuộc tính id vào trong table 
def get_tmdb_id(title: str, release_date): 
    t  = release_date.year
    movie = search.movies(title, release_year=t)

    for j in movie['results']: 
        if j['release_date'] == release_date.strftime('%Y-%m-%d'): 
            return j['id']
        
    return None



In [9]:
data_set = pd.DataFrame(columns=['English title', 'Released', 'TMDB_id'])


for i in range(2009, 2013): 
    print(f"Processing year {i}")
    raw_data = get_films_on_wiki(i)
    
    data_clean = change_release_year(raw_data, i)
    data_clean['TMDB_id'] = data_clean[['English title', 'Released']].apply(lambda x: get_tmdb_id(x[0],  x[1]), axis = 1) 
    data_set = pd.concat([data_set, data_clean])

data_set.head()


Processing year 2009


C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_month = dct[x[1]]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return str(year)  + '-' + str(num_month) + '-' + x[0]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\3939438557.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_clean['TMDB_id'] = 

Processing year 2010


C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_month = dct[x[1]]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return str(year)  + '-' + str(num_month) + '-' + x[0]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\3939438557.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_clean['TMDB_id'] = 

Processing year 2011


C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_month = dct[x[1]]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return str(year)  + '-' + str(num_month) + '-' + x[0]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\3939438557.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_clean['TMDB_id'] = 

Processing year 2012


C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_month = dct[x[1]]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\303362258.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return str(year)  + '-' + str(num_month) + '-' + x[0]
C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\3939438557.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_clean['TMDB_id'] = 

,English title,Released,TMDB_id
0,20th Century Boys 2: The Last Hope,2009-01-31,NaN
1,20th Century Boys 3: Redemption,2009-08-29,NaN
2,Air Doll,2009-05-14,NaN
3,Amalfi: Rewards of the Goddess,2009-07-18,46194.0
4,April Bride,2009-05-09,55749.0


In [10]:
data_set

,English title,Released,TMDB_id
0,20th Century Boys 2: The Last Hope,2009-01-31,NaN
1,20th Century Boys 3: Redemption,2009-08-29,NaN
2,Air Doll,2009-05-14,NaN
3,Amalfi: Rewards of the Goddess,2009-07-18,46194.0
4,April Bride,2009-05-09,55749.0
...,...,...,...
164,Good Morning Everyone!,2012-12-15,350786.0
165,One Piece: Film Z,2012-12-15,176983.0
166,Yōkai Ningen Bem,2012-12-15,204086.0
167,Odayaka na Nichijo,2012-12-22,153789.0


In [11]:
check =  data_set.dropna(axis = 0)
check

,English title,Released,TMDB_id
3,Amalfi: Rewards of the Goddess,2009-07-18,46194.0
4,April Bride,2009-05-09,55749.0
5,Arceus Chōkoku no Jikū e,2009-07-18,39057.0
6,Asahiyama Zoo Story- Penguins in the Sky,2009-02-07,328093.0
8,Cho Kamen Rider Den-O & Decade Neo Generations...,2009-05-01,178848.0
...,...,...,...
160,Sono Yoru no Samurai,2012-11-17,135799.0
164,Good Morning Everyone!,2012-12-15,350786.0
165,One Piece: Film Z,2012-12-15,176983.0
166,Yōkai Ningen Bem,2012-12-15,204086.0


In [12]:
check['TMDB_id'] = check['TMDB_id'].astype(int)

C:\Users\Asus\AppData\Local\Temp\ipykernel_18024\3804415964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['TMDB_id'] = check['TMDB_id'].astype(int)


In [13]:
check.head()

,English title,Released,TMDB_id
3,Amalfi: Rewards of the Goddess,2009-07-18,46194
4,April Bride,2009-05-09,55749
5,Arceus Chōkoku no Jikū e,2009-07-18,39057
6,Asahiyama Zoo Story- Penguins in the Sky,2009-02-07,328093
8,Cho Kamen Rider Den-O & Decade Neo Generations...,2009-05-01,178848


In [14]:
check.reset_index(inplace=True)

In [ ]:
check.to_csv('raw_jan_09_12.csv')